# Article notes

[Multi-armed Bandit Models for the
Optimal Design of Clinical Trials: Benefits
and Challenges](https://arxiv.org/pdf/1507.08025.pdf),  
_Sofía S. Villar, Jack Bowden and James Wason, 2015_

## 1. Introduction

- **Goal**: study algorithms for the **M**ulti **A**rmed **B**andit **P**roblem, using a Bayesian approach
- Criteria:
    - _exploration_ or _learning_: identify the best arm
    - _exploitation_ or _earning_: maximize expected total reward


## 2. Bayesian Bernouilli MABP

- Each arm $k = 1, \dots, K \sim Bernouilli(p_k)$
- Prior on $p_k \sim Beta(s_{k,0}, f_{k,0})$
- Posterior, having observed $S_{k,t} = s_{k,t}$ successes and $F_{k,t} = f_{k,t}$ failures: $$p_k^{(t)} \sim Beta(s_{k,0} + s_{k,t}, \, f_{k,0} + f_{k,t})$$

Formulated as **Markov Control Process** (from the Optimal Control litterature):

> - _state space_: $\{(s_{k,0} + S_{k,t}, \, f_{k,0} + F_{k,t}) \in \mathbb{N}^2_+ \, | \, S_{k,t} + F_{k,t} \leq t \quad \forall t \in 1 \dots T\}$

> - _action space_: $\{\mathbf{a}_t = (a_{k,t})_k \, | \, a_{k,t} = \mathbb{1}_{\text{arm} \, k \, \text{used at} \, t}\}$  
    > Note: at each $t$, $\sum_k a_{k, t} = 1$

> - _transition law_: if arm $k$ is drawn, 
    $$\mathbb{P}(S_{k, t+1} = s_{k, t} + 1) = \frac{s_{k,0} + s_{k,t}}{s_{k,0} + s_{k,t} + f_{k,0} + f_{k,t}}$$
    $$\mathbb{P}(F_{k, t+1} = f_{k, t} + 1) = \frac{f_{k,0} + f_{k,t}}{s_{k,0} + s_{k,t} + f_{k,0} + f_{k,t}}$$

Objective: construct an optimal _policy_ (i.e. a set of actions $\{\mathbf{a}_t\}_t$), according to the objective function
    $$\mathbb{E}^\pi \Big[ \sum_{t=0}^{T-1} \sum_{k=1}^K d^t \frac{s_{k,0} + S_{k,t}}{s_{k,0} + S_{k,t} + f_{k,0} + F_{k,t}} \cdot a_{k,t} \quad | \quad (x_{k,0}) \Big]$$
    
where we know $\mathbb{E}^\pi \bigg[\frac{s_{k,0} + S_{k,t}}{s_{k,0} + S_{k,t} + f_{k,0} + F_{k,t}}\bigg]$ is the expected reward at timestep $t$ for drawing arm $k$

To ease notations, let us define $\lambda_{k,t} = \frac{s_{k,0} + s_{k,t}}{s_{k,0} + s_{k,t} + f_{k,0} + f_{k,t}}$ and $\Lambda_{k,t} = \frac{s_{k,0} + S_{k,t}}{s_{k,0} + S_{k,t} + f_{k,0} + F_{k,t}}$.

_Matrix notation_ ($\Lambda$ and $\mathbf{a}$ are of size (K x T), $\gamma$ T-vector): $$\Lambda = (\Lambda_{k,t})_{(k,t) \in [[1, K]] \times [[0, T-1]]} \quad \mathbf{a} = (\mathbf{a}_t)_{t \in [[0, T-1]]} \quad \gamma = (d^t)_{t \in [[0, T-1]]}$$

Then, the "Total Discounted Expected Reward" is : $\mathbb{E}^\pi \big(\gamma' \Lambda' \mathbf{a} \, | \, \mathbf{x} \big)$

Introduces the _regret_: $$\rho = T\max_k p_k - \mathbb{E}^\pi \big(\mathbf{1}_T Y' \mathbf{a} \big)$$

## 3. Infinite-horizon case

__Bellman optimality equation__:

if we note the optimal value at date $T$: $v^*_T = \max_\pi \mathbb{E}^\pi \big(\gamma' \Lambda' \mathbf{a} \, | \, \mathbf{x} \big)$, then we have: $$v^*_T = \max_k \big( \lambda_{k,t} + d \, \mathbb{E}^\pi ( v^*_{T + 1} ) \big)$$

__Gittins index__:

$$\mathcal{G}_k(x_{k,t}) = \sup_{\tau \geq 1} \mathbb{E}^\pi \Big( \frac{\sum_{i=0}^{\tau - 1} \mathcal{R}(X_{k, t+i}, 1) \, d^i}{\sum_{i=0}^{\tau - 1} \mathcal{C}(X_{k, t+i}, 1) d^i} \quad | \; x_{k,t} \Big)$$

...or in our case:

$$\mathcal{G}_k(x_{k,t}) = \sup_{\tau \geq 1} \mathbb{E}^\pi \Big( \frac{\sum_{i=0}^{\tau - 1} \Lambda_{k, t+i} \, d^i}{\sum_{i=0}^{\tau - 1} d^i} \quad | \; x_{k,t} \Big)$$

_Efficient computation method_: